In [ ]:
import networkx  # Permite trabajar con grafos
import pgmpy.models as pgmm  # Modelos gráficos de probabilidad
import pgmpy.factors as pgmf  # Tablas de probabilidades condicionales y
                              # potenciales de probabilidad
import pgmpy.inference as pgmi  # Inferencia probabilística exacta